In [68]:
import os, sys, email, datetime, pprint, re, time, html, pickle
import numpy as np
import pandas as pd
import nltk
from nltk.metrics import *

pd.options.display.max_colwidth = 1000

In [69]:
### 自然言語処理
import spacy 
nlp = spacy.load("en_core_web_sm")

from spacy.pipeline import Sentencizer
sentencizer = Sentencizer()
nlp.add_pipe(sentencizer)

In [70]:
with open('/Users/taroaso/myprojects/OpenIE/trec/output/tagme.pickle', mode="rb") as f:
    tagme_result = pickle.load(f)

with open('/Users/taroaso/myprojects/OpenIE/trec/output/ner.pickle', mode="rb") as f:
    ner_result = pickle.load(f)

with open('/Users/taroaso/myprojects/OpenIE/trec/output/type_translation.pickle', mode="rb") as f:
    translation = pickle.load(f)

with open('/Users/taroaso/myprojects/OpenIE/trec/output/openie_result.pickle', mode="rb") as f:
    openie = pickle.load(f)

In [91]:
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD
BASE = Namespace("http://www.kde.cs.tsukuba.ac.jp/~aso/w3c-email/")
EMAIL = Namespace("http://www.w3.org/2000/10/swap/pim/email#")
OLIA = Namespace("http://purl.org/olia/olia.owl#")
NERD = Namespace("http://nerd.eurecom.fr/ontology#")
NIF = Namespace("http://persistence.uni-leipzig.org/nlp2rdf/ontologies/nif-core#")
ITSRDF = Namespace("https://www.w3.org/2005/11/its/rdf#")
WD = Namespace("http://www.wikidata.org/entity/")

In [109]:
# RDFグラフ
g = Graph()
g.bind('',BASE)
g.bind('schema',SDO)
g.bind('foaf',FOAF)
g.bind('owl',OWL)
g.bind('rdf',RDF)
g.bind('rdfs',RDFS)
g.bind('olia',OLIA)
g.bind('nerd',NERD)
g.bind('nif',NIF)
g.bind('itsrdf',ITSRDF)
g.bind('wd',WD)

In [110]:
def openie2rdf(g,openie_result,tagme_result): #g:RDFグラフ, openie_resultの結果，tagme_result:tagmeの結果
    g = g
    openie = openie_result
    tagme_result = tagme_result
    url_pattern = re.compile(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+')
    email_pattern = re.compile(r'[\w\-._]+@[\w\-._]+\.[A-Za-z]+')
    # メールごとにopenIEの結果を取り出す
    for uid, results in openie.items():
        #URI参照される表記に変換する
        emailmessage = re.sub(r'[^a-zA-Z_0-9]','_',uid)
        statementId = BNode()
        # openIEの結果とエンティティの抽出結果があるときのみ,処理を継続する
        if results == [] or tagme_result[uid] is None:
            pass
        else:
            # メールの文ごとにopenIEの結果を取り出す
            for result in results:
                sentence = result['sentence']
                sent_start = result['start']
                facts = result['facts']
                # トリプルを取り出す
                for fact in facts:
                    s = fact['subject']
                    p = fact['predicate']
                    p = re.sub(r'[^a-zA-Z_0-9]','_',p)
                    o = fact['object']
                    # subjectとobjectのオフセットを取得する
                    s_match = re.search(re.escape(s),sentence)
                    o_match = re.search(re.escape(o),sentence)
                    if s_match is None: # sがsentenceにマッチしない文字列なら次のfactへ
                        pass
                    else: # sがsentenceにマッチする場合, RDF化を進める
                        s_start = s_match.start() + sent_start
                        s_end = s_match.end() + sent_start
                        # tagmeのアノテーション結果を走査する
                        for ann in tagme_result[uid]['annotations']:         
                            # トリプルの主語とspot, start, endが一致する場合，トリプル（主語，メンション，検出されたスポットのリソース）作成
                            if s == ann['spot'] and s_start == ann['start'] and s_end == ann['end'] and ann['link_probability'] >= 0.3:
                                #URI参照される表記に変換する
                                s = re.sub(r'[^a-zA-Z_0-9]','_',s)
                                offset_based_string = emailmessage + '#offset_' + str(s_start) + '_' + str(s_end)
                                g.add((BASE[offset_based_string],OWL.sameAs,BASE[s]))
                                if o_match is None:
                                    if url_pattern.search(o):
                                        o = URIRef(o)
                                        g.add((BASE[s],BASE[p],o))
                                        g.add((BASE[emailmessage], SDO.mentions, statementId))
                                        g.add((statementId, RDF.type, RDF.Statement))
                                        g.add((statementId, RDF.subject,BASE[s]))
                                        g.add((statementId, RDF.predicate, BASE[p]))
                                        g.add((statementId, RDF.object, o))
                                    elif email_pattern.search(o):
                                        o = 'mailto:'+ o
                                        o = URIRef(o)
                                        g.add((BASE[s],BASE[p],o))
                                        g.add((BASE[emailmessage], SDO.mentions, statementId))
                                        g.add((statementId, RDF.type, RDF.Statement))
                                        g.add((statementId, RDF.subject,BASE[s]))
                                        g.add((statementId, RDF.predicate, BASE[p]))
                                        g.add((statementId, RDF.object, o))                                      
                                    else:
                                        o = re.sub(r'[^a-zA-Z_0-9]','_',o)
                                        oie_triples.append((s,p,o))
                                        g.add((BASE[s],BASE[p],Literal(o,lang='en')))
                                        g.add((BASE[emailmessage], SDO.mentions, statementId))
                                        g.add((statementId, RDF.type, RDF.Statement))
                                        g.add((statementId, RDF.subject,BASE[s]))
                                        g.add((statementId, RDF.predicate, BASE[p]))
                                        g.add((statementId, RDF.object, Literal(o,lang='en')))
                                else:
                                    o_start = o_match.start() + sent_start
                                    o_end = o_match.end() + sent_start
                                    resource_check = False
                                    # tagmeのアノテーション結果を走査する
                                    for ann in tagme_result[uid]['annotations']:
                                        #　トリプルの目的語と一致するspotがある場合，（目的語，メンション，検出されたスポットのリソース）作成
                                        if o == ann['spot'] and o_start == ann['start'] and o_end == ann['end'] and ann['link_probability'] >= 0.3:
                                            o = ':' + re.sub(r'[^a-zA-Z_0-9]','_',o)
                                            offset_based_string = emailmessage + '#offset_' + str(o_start) + '_' + str(o_end)
                                            g.add((BASE[offset_based_string],OWL.sameAs,BASE[o]))
                                            g.add((BASE[s],BASE[p],BASE[o]))
                                            g.add((BASE[emailmessage], SDO.mentions, statementId))
                                            g.add((statementId, RDF.type, RDF.Statement))
                                            g.add((statementId, RDF.subject,BASE[s]))
                                            g.add((statementId, RDF.predicate, BASE[p]))
                                            g.add((statementId, RDF.object, BASE[o]))
                                            resource_check = True
                                    #openIEのトリプル作成
                                    if resource_check == True:
                                        continue
                                    else:
                                        if url_pattern.search(o):
                                            o = URIRef(o)
                                            g.add((BASE[s],BASE[p],o))
                                            g.add((BASE[emailmessage], SDO.mentions, statementId))
                                            g.add((statementId, RDF.type, RDF.Statement))
                                            g.add((statementId, RDF.subject,BASE[s]))
                                            g.add((statementId, RDF.predicate, BASE[p]))
                                            g.add((statementId, RDF.object, o))
                                        elif email_pattern.search(o):
                                            o = 'mailto:'+ o
                                            o = URIRef(o)
                                            g.add((BASE[s],BASE[p],o))
                                            g.add((BASE[emailmessage], SDO.mentions, statementId))                                        
                                            g.add((statementId, RDF.type, RDF.Statement))
                                            g.add((statementId, RDF.subject,BASE[s]))
                                            g.add((statementId, RDF.predicate, BASE[p]))
                                            g.add((statementId, RDF.object, o))
                                        else:
                                            o = re.sub(r'[^a-zA-Z_0-9]','_',o)
                                            oie_triples.append((s,p,o))
                                            g.add((BASE[s],BASE[p],Literal(o,lang='en')))
                                            g.add((BASE[emailmessage], SDO.mentions, statementId))
                                            g.add((statementId, RDF.type, RDF.Statement))
                                            g.add((statementId, RDF.subject,BASE[s]))
                                            g.add((statementId, RDF.predicate, BASE[p]))
                                            g.add((statementId, RDF.object, Literal(o,lang='en')))

In [111]:
openie2rdf(g,openie,tagme_result)

In [112]:
print(g.serialize(format="turtle").decode("utf-8"))

@prefix : <http://www.kde.cs.tsukuba.ac.jp/~aso/w3c-email/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix schema: <https://schema.org/> .

:000001c4264e_f6dab750_440bc650_tversdata schema:mentions [ a rdf:Statement ;
            rdf:object "also_locally"@en,
                "even_faster"@en,
                "job"@en,
                "most_cases"@en,
                "search_engine"@en ;
            rdf:predicate :do,
                :do_better,
                :do_job,
                :do_job_in,
                :is ;
            rdf:subject :Google ] .

<http://www.kde.cs.tsukuba.ac.jp/~aso/w3c-email/000001c4264e_f6dab750_440bc650_tversdata#offset_449_455> owl:sameAs :Google .

<http://www.kde.cs.tsukuba.ac.jp/~aso/w3c-email/000001c4264e_f6dab750_440bc650_tversdata#offset_885_891> owl:sameAs :Google .

:000401c33289_e4869740_6401a8c0_handsontech schema:mentions [ a rdf:Statement ;
            rdf:object "static"@e

In [113]:
len(g)

944

In [114]:
# turtle形式でファイル出力
g.serialize(destination='/Users/taroaso/myprojects/OpenIE/trec/output/oie_triples.ttl', format='turtle')